In [1]:
# lets start by getting pandas, regex, and numpy
import pandas as pd
import re
import numpy as np
import requests
from bs4 import BeautifulSoup
import helper as helper

# 2. Background # 
This is the stuff we need to make this work. </br>
So we need the relevant dataframes, and the relevant functions. </br>
The functions are: 
1) 'ingredient_divider' which takes a string and returns the percengtage of ingredietns which are covered
2) 'matching_ingredient' which takes a url, and updates the recipes dataframe with the ingredients and the percentage of ingredients covered
3) 'matching_ingredient_df' which takes a df, and updates the recipes dataframe with the ingredients and the percentage of ingredients covered
4) 'check_fridge_availability' which takes an ingredient name and amount needed, and returns the amount available in the fridge
5) 'calculate_cost_and_remaining' which takes a row and returns the cost and remaining amount of ingredients needed
6) 'calculate_cost_of_urls' which takes a list of urls and returns the total cost of the ingredients needed

In [3]:
recipes = pd.read_csv('recipes.csv')
ingredients_df = pd.read_csv('ingredients_df.csv')
pantry = pd.read_csv('pantry.csv')
# lets go to the download folder and import the ingredients_3.csv file
conversion = pd.read_csv('conversion.csv')
fridge = pd.read_csv('fridge.csv')

In [4]:
fridge = pd.read_clipboard()
fridge.columns = ['ingredients', 'amounts']
fridge = fridge[fridge.amounts > 0]

fridge

,ingredients,amounts
0,løg,3.0
1,hvidløg,8.0
2,rødvin,740.0
3,olivenolie,970.0
8,grøntsagsbouillon,150.0
9,soja,200.0
11,mango,125.0
12,rødløg,6.5
13,jalapenos,45.0
15,lime,4.0


In [5]:
fridge.loc[len(fridge)] = ['æg', 5]

In [8]:
# now lets merge the purchase_unit and the veggie average on to the fridge 
fridge = pd.merge(fridge, ingredients_df[['ingredients', 'purchase_unit', 'veggie_average']], on='ingredients', how='left')

In [13]:
# now lets copy all this in to a clipboard where we can base into a text emssage, where we send the amount we have left, the veggeige average followed by the purchase unit and then the ingredient
fridge['text'] = fridge['amounts'].astype(str) + ', tilbage ' + round(fridge['veggie_average'], 2).astype(str) + ', normalt ' + fridge['purchase_unit'].astype(str) + ' ' + fridge['ingredients'].astype(str)
# now lets copy all that into one string
fridge_text = fridge['text'].str.cat(sep='\n')
fridge_text.replace('nan', 'ved ikke')
import pyperclip    
pyperclip.copy(fridge_text)


In [3]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse, urljoin
from tqdm import tqdm


def urls_on_same_domain(url, max_urls=400):
    """
    Counts the number of unique URLs on the same domain as the given URL,
    iterating through links until no more are found or a max limit is reached.
    The progress is displayed using tqdm.

    Args:
    url (str): The starting URL to fetch and analyze.
    max_urls (int): Maximum number of URLs to fetch.

    Returns:
    int: The total number of unique URLs found on the same domain.
    """
    visited_urls = set()
    urls_to_visit = set([url])
    domain = urlparse(url).netloc

    # Common user-agent string of a web browser
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }

    with tqdm(total=max_urls, desc=f"Analyzing URLs currently found {len(visited_urls)}, and have {len(urls_to_visit)} to go", unit="url") as pbar:
        while len(urls_to_visit) != 0 and len(visited_urls) < max_urls:
            current_url = urls_to_visit.pop()
            try:
                response = requests.get(current_url, headers=headers)
                response.raise_for_status()

                soup = BeautifulSoup(response.content, 'html.parser')
                anchors = soup.find_all('a', href=True)

                for anchor in anchors:
                    href = anchor['href']
                    full_url = urljoin(current_url, href)
                    href_domain = urlparse(full_url).netloc

                    if href_domain == domain and full_url not in visited_urls and '#comment' not in full_url and '#respond' not in full_url:
                        urls_to_visit.add(full_url)

                visited_urls.add(current_url)
                pbar.update(1)
                pbar.set_description(f"Analyzing URLs currently found {len(visited_urls)}, and have {len(urls_to_visit)} to go")
            except requests.RequestException:
                # Handle exceptions for requests
                continue

    return visited_urls

def get_text_from_url(url):
    """
    Fetches the content from a given URL and returns the text separated by lines.
    If the URL gets redirected more than twice, returns 'redirect link'.

    Args:
    url (str): The URL from which to fetch the content.

    Returns:
    str: A string containing the text from the URL, separated by lines, or 'redirect link'.
    """
    # Common user-agent string of a web browser
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }

    # Send a GET request to the URL with the user-agent header and redirection allowed
    response = requests.get(url, headers=headers, allow_redirects=True)
    
    # Check the number of redirects
    if len(response.history) > 2:
        return 'redirect link'
    
    # Raise an exception if the request was unsuccessful
    response.raise_for_status()

    # Parse the content of the request with BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    # Extract and return the text, separated by lines
    return '\n'.join(soup.stripped_strings)

url = 'https://vegetariskhverdag.dk/opskrifter/'

test_urls = urls_on_same_domain(url, max_urls=1000)

# now lets write them to a file called 'dovne_vegetar.txt'
with open('dovne_vegetar.txt', 'w') as f:
    for url in tqdm(test_urls):
        f.write(url + '\n')

Analyzing URLs currently found 43, and have 233 to go:   7%|▋         | 70/1000 [00:52<10:25,  1.49url/s]Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Analyzing URLs currently found 55, and have 247 to go:   9%|▉         | 92/1000 [01:08<11:07,  1.36url/s]Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Analyzing URLs currently found 111, and have 342 to go:  19%|█▉        | 192/1000 [02:29<12:35,  1.07url/s]Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Analyzing URLs currently found 114, and have 347 to go:  20%|██        | 200/1000 [02:34<09:58,  1.34url/s]Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Analyzing URLs currently found 144, and have 378 to go:  25%|██▌       | 251/1000 [03:15<09:26,  1.32url/s]Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Analyzing URLs currently found 162, and have 406 

In [2]:
test_urls

918